# Lab 10 Task 1 - Using VGG16 on a dataset

### SAHIL AMRITKAR
### 20BRS1180  


In [ ]:
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images.shape
len(train_labels)
train_labels
test_images.shape
len(test_labels)
test_labels

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [8]:
from tensorflow import keras
import tensorflow as tf

(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

validation_images, validation_labels = train_images[:50], train_labels[:50]
train_images, train_labels = train_images[50:500], train_labels[50:500]

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (224,224))
    return image, label

train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
test_ds = (test_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32, drop_remainder=True))

Training data size: 450
Test data size: 10000
Validation data size: 50


In [9]:
from tensorflow import keras
import tensorflow as tf

In [10]:
from keras.applications.vgg16 import VGG16

model =VGG16(
    include_top=True,
    #weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=max,
    classes=1000,
    classifier_activation='softmax'
)
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.RMSprop(), metrics=['accuracy'])

In [12]:
history=model.fit(train_ds,
          epochs=2,
          validation_data=validation_ds,)

Epoch 1/2


2023-03-01 20:08:13.851654: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


14/14 [==============================] - ETA: 0s - loss: 19685402.0000 - accuracy: 0.0960

2023-03-01 20:08:34.963758: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


14/14 [==============================] - 22s 1s/step - loss: 19685402.0000 - accuracy: 0.0960 - val_loss: 3.3335 - val_accuracy: 0.0625
Epoch 2/2
14/14 [==============================] - 18s 1s/step - loss: 2.5206 - accuracy: 0.0960 - val_loss: 2.5214 - val_accuracy: 0.0625


In [13]:
model.evaluate(test_ds)

312/312 [==============================] - 98s 313ms/step - loss: 2.4525 - accuracy: 0.1016


[2.4524636268615723, 0.1015625]